# Falsify Neural Network Controlled F16 GCAS

Based on demo in https://github.com/caffett/falsify_neural_f16gcas

In [ ]:
import os
import sys

import numpy as np
from numpy import pi, deg2rad
from skopt import gp_minimize

# CSAF Imports
import csaf.system as csys
import csaf.config as cconf

# Analsis Objects/Functions
from falsify_bopt import F16SimulationFunction, EvaluatedFunction, attack, initial_space

In [ ]:
# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_shield_config.toml")

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

In [ ]:
def ground_collision_condition(cname, outs):
        """ground collision premature termnation condition"""
        return cname == "plant" and outs["states"][11] <= 0.0


def gcas_simulation(initial_state, initial_time, tmax=35):
    """simulate GCAS system"""
    tspan = (initial_time, tmax)
    my_system.reset()
    my_system.set_state("plant", initial_state[:13])
    my_system.set_state("controller", initial_state[13:])
    trajs, passed = my_system.simulate_tspan(tspan, show_status=True,
                                terminating_conditions=ground_collision_condition,
                                return_passed=True)
    tlen = min(len(trajs["plant"].states), len(trajs["controller"].states))
    return trajs["plant"].times, np.hstack((trajs["plant"].states[:tlen], trajs["controller"].states[:tlen])), passed


In [ ]:
## simulate
l, h = initial_space()
initial_state = np.random.uniform(l, h)
times, states, passed = gcas_simulation(initial_state, 0)

In [ ]:
## attack
attack_traj = np.array(states)[0:1]
delta = (h - l)/2
attack_spaces = [attack_traj[0]-delta, attack_traj[0]+delta]
unsafe_states = attack(EvaluatedFunction(F16SimulationFunction(0, gcas_simulation)), attack_spaces, n_calls=10)

In [ ]:
## unbind/disconnect system
my_system.unbind()

In [ ]:
## print number of unsafe states found
print(f"Number of unsafe states found: {len(unsafe_states)}")